In [2]:
import re

N = 10
M = 10
found = [[0] * M for i in range(0,N)]
cntFound = 0
A, B, C = [], [], []
for ln in reversed(open("result_start_10.10_01","r").readlines()):
    if cntFound == N * M:
        break
    if ln.startswith("0 / L / Y /"):

        cell = ln.split('/')[4]
        p = re.compile("opinion(.*)\([0-9]+\)")
        s = p.findall(cell)[0]
        x, y, z = int(s[1]), int(s[3]), int(s[5])
        if(z == 0 and found[x][y] == 0):
            cntFound += 1
            found[x][y] = 1
            
            val  = ln.split('/')[6]
            #print([x,y])
            val = float(val.strip())
            
            if(val < -1): 
                A.append(val)
            elif(val >= -1 and val <= 1):
                B.append(val)
            elif(val > 1):
                C.append(val)
print(len(A))
print(len(B))
print(len(C))


28
22
50
